In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [2]:
url="https://www.airlinequality.com/airline-reviews/british-airways/page/"
total_pages=353

In [3]:
data={
    "Review Header":[],
    "Review Text":[],
    "User":[],
    "Date Published":[],
    'Aircraft':[],
    "Type Of Traveller":[],
    "Seat Type":[],
    "Route":[],
    "Date Flown":[],
    "Seat Comfort":[],
    "Cabin Staff Service":[],
    "Food & Beverages":[],
    "Inflight Entertainment":[],
    "Ground Service":[],
    "Wifi & Connectivity":[],
    "Value For Money":[],
    "Recommended":[],
}

In [4]:
def get_missing_stats(present_stats):
    all_stats = ['Aircraft', 'Type Of Traveller', 'Seat Type', 'Route', 'Date Flown',
                 'Seat Comfort', 'Cabin Staff Service', 'Food & Beverages', 'Inflight Entertainment',
                 'Ground Service', 'Wifi & Connectivity', 'Value For Money', 'Recommended']
    missing_stats = []
    for stat in all_stats:
        if stat not in present_stats:
            missing_stats.append(stat)
    return missing_stats

In [5]:
for i in range(1,total_pages+1):
    url=url+str(i)
    response=requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    # Find all the review blocks on the page
    reviews= soup.find_all('div', class_='body')
    for review in reviews:
        #Extract review header
        data['Review Header'].append(review.find("h2", class_="text_header").text.strip())
        #Extract user name
        data["User"].append(review.find("span", itemprop="name").text.strip())
        #Extract date when the review was written
        data['Date Published'].append(review.find("time", itemprop="datePublished").text.strip())
    
        #Extract the review text and verification status
        data['Review Text'].append(review.find("div", itemprop="reviewBody").text.strip())
    
        #Check For other available stats
        #Note: Total stats which could be present are 13
        stats = review.find_all("tr")
        present_stats=[]
        for stat in stats:
            label = stat.find("td", class_="review-rating-header").text.strip()
            temp=stat.find("td",class_="review-rating-stars stars")
            if temp:
                value=len(temp.find_all("span",class_="star fill"))
            else:
                value = stat.find("td", class_="review-value").text.strip()
            #Extract the stats present in the review
            data[label].append(value)
            
            present_stats.append(label)
        for x in get_missing_stats(present_stats):
            #Substitute NaN in place of missing Stats
            data[x].append(np.NaN)

In [6]:
#convert to dataframe
df=pd.DataFrame(data)

In [7]:
df.head(30)

,Review Header,Review Text,User,Date Published,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Inflight Entertainment,Ground Service,Wifi & Connectivity,Value For Money,Recommended
0,"""trained to give you the runaround""",✅ Trip Verified | Worst experience of my life...,T Cayle,17th April 2023,NaN,Family Leisure,Economy Class,London to Denver,March 2023,2.0,5.0,2.0,3.0,4,1.0,1,no
1,"""they only had one choice of meal""",✅ Trip Verified | Due to code sharing with Ca...,Andrew Pybus,17th April 2023,NaN,Solo Leisure,Economy Class,Hong Kong to London,April 2023,2.0,1.0,1.0,1.0,2,1.0,2,no
2,"""relentless BA cost cutting""",✅ Trip Verified | LHR check in was quick at t...,M Edwards,16th April 2023,A320,Business,Business Class,Heathrow to Malaga,April 2023,3.0,3.0,4.0,NaN,3,1.0,2,no
3,"""I wouldn't recommend British Airways""",✅ Trip Verified | I wouldn't recommend Britis...,Luis Rojas,15th April 2023,NaN,Family Leisure,Economy Class,Santorini to Gatwick,March 2023,4.0,4.0,NaN,1.0,3,3.0,1,no
4,"""Absolutely horrible experience""",✅ Trip Verified | Absolutely horrible experie...,C Maire,15th April 2023,NaN,Solo Leisure,Economy Class,Madrid to Belfast via London,January 2023,3.0,3.0,2.0,NaN,1,NaN,1,no
5,"""This is the worst airline""",Not Verified | This is the worst airline. Not...,Megan Campbell,14th April 2023,NaN,Business,Economy Class,Dublin to London,April 2023,1.0,1.0,1.0,1.0,1,1.0,1,no
6,"""never fly British Airways again""",✅ Trip Verified | I will never fly British Ai...,L Horten,13th April 2023,NaN,Couple Leisure,Business Class,London to Venice,March 2023,1.0,2.0,2.0,1.0,2,NaN,2,no
7,"""seats were cramped and uncomfortable""",✅ Trip Verified | Worst aircraft I have ever ...,L Keane,12th April 2023,A350,Couple Leisure,Economy Class,Las Vegas to London,April 2023,1.0,1.0,1.0,4.0,4,1.0,1,no
8,"""I enjoyed my flight""",✅ Trip Verified | I enjoyed my flight. The bo...,Brett Hardy,11th April 2023,A320,Solo Leisure,Business Class,Faro to Heathrow,April 2023,5.0,5.0,5.0,NaN,3,NaN,5,yes
9,"""found the food menu rather odd""",✅ Trip Verified | Why do you make it so hard? ...,P Jones,10th April 2023,Boeing 777,Couple Leisure,Business Class,London to Atlanta,April 2023,4.0,3.0,2.0,3.0,2,3.0,3,no


In [8]:
df.tail()

,Review Header,Review Text,User,Date Published,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Inflight Entertainment,Ground Service,Wifi & Connectivity,Value For Money,Recommended
3415,"""This is the worst airline""",Not Verified | This is the worst airline. Not...,Megan Campbell,14th April 2023,NaN,Business,Economy Class,Dublin to London,April 2023,1.0,1.0,1.0,1.0,1,1.0,1,no
3416,"""never fly British Airways again""",✅ Trip Verified | I will never fly British Ai...,L Horten,13th April 2023,NaN,Couple Leisure,Business Class,London to Venice,March 2023,1.0,2.0,2.0,1.0,2,NaN,2,no
3417,"""seats were cramped and uncomfortable""",✅ Trip Verified | Worst aircraft I have ever ...,L Keane,12th April 2023,A350,Couple Leisure,Economy Class,Las Vegas to London,April 2023,1.0,1.0,1.0,4.0,4,1.0,1,no
3418,"""I enjoyed my flight""",✅ Trip Verified | I enjoyed my flight. The bo...,Brett Hardy,11th April 2023,A320,Solo Leisure,Business Class,Faro to Heathrow,April 2023,5.0,5.0,5.0,NaN,3,NaN,5,yes
3419,"""found the food menu rather odd""",✅ Trip Verified | Why do you make it so hard? ...,P Jones,10th April 2023,Boeing 777,Couple Leisure,Business Class,London to Atlanta,April 2023,4.0,3.0,2.0,3.0,2,3.0,3,no


In [9]:
df.to_csv("data/British-Airways-Reviews.csv")